## 1. Init

In [1]:
import os
import os.path as op
import nltk
import re
import numpy as np
import pandas as pd
from collections import Counter

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from func import *

## 2. Feature Engineering
### 2.1 Blogs

In [ ]:
#Mine POS patterns
tagList = [
    'NN', 'CC', 'LS', 'PDT', 'POS', 'SYM', 'NNS', 'NNP', 'NNPS', 'FW', 'CD',
    'JJ', 'JJR', 'JJS', 'IN', 'TO', 'DT', 'EX', 'PRP', 'PRP$', 'WDT', 'WP',
    'WP$', 'MD', 'VB', 'VBZ', 'VBP', 'VBD', 'VBN', 'VBG', 'RB', 'RBR', 'RBS',
    'RP', 'WRB', 'UH', '.'
]

for gender in [0, 1]:
    if gender == 0:
        txtDir = './postprocess_blogs/blogs/female/'
    else:
        txtDir = './postprocess_blogs/blogs/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in range(0, len(blogs_gender)):
        m = blogs_gender[i]
        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        for file in os.listdir(txtDir + m):
            text = gettext(txtDir + m + '/' + file)
            sentences = nltk.sent_tokenize(text)
            CorpusPOS(sentences)

infile = open('CorpusPOS.txt', 'r')
cPOS = infile.readlines()
infile.close()
(a, b, c, d, e) = calc_probabilities(cPOS)
q1_output(a, b, c, d, e)

Prob = {}
infile = open('probabilities.txt', 'r')
prob_text = infile.readlines()

for sentence in prob_text:
    keyValPair = sentence.split(":")
    Prob[keyValPair[0]] = float(keyValPair[1][:-1])

infile.close()

posFeatures = minePOSPats(cPOS)

In [ ]:
#Feature extract
names = []
F_features = []
GRF_features = []
WC_features = []
POS_features = []
labels = []

for gender in [0, 1]:
    if gender == 0:
        txtDir = './postprocess_blogs/blogs/female/'
    else:
        txtDir = './postprocess_blogs/blogs/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in range(0, len(blogs_gender)):
        m = blogs_gender[i]
        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        for file in os.listdir(txtDir + m):
            name = txtDir + m + '/' + file
            text = gettext(name)
            words = getwords(text)
            sentences = nltk.sent_tokenize(text)
            words_s = split(words)
            tags = [nltk.pos_tag(word) for word in words]
            tags_s = split(tags)
            words_l = wordlemmatize(tags_s)

            F_feature = F_measure(tags_s)
            GRF_feature = Gender_Preferential_Features(words_l)
            WC_feature = Word_Classes_Feature(words_l)

            textTags = ""
            for word, tag in tags_s:
                if tag in tagList:
                    textTags = textTags + tag + " "

            POS_feature = []

            for feature in posFeatures:
                if feature in textTags:
                    POS_feature.append(1)
                else:
                    POS_feature.append(0)

            names.append(name)
            F_features.append(F_feature)
            GRF_features.append(GRF_feature)
            WC_features.append(WC_feature)
            POS_features.append(POS_feature)
            labels.append(gender)

In [ ]:
#Get single
def getsingle(features, n):
    single = []
    for item in features:
        single.append(item[n])
    return single


WC_features_l = []
for i in range(len(WC_features[0])):
    n = i
    WC_features_l.append(getsingle(WC_features, n))

GRF_features_l = []
for i in range(len(GRF_features[0])):
    n = i
    GRF_features_l.append(getsingle(GRF_features, n))

POS_features_l = []
for i in range(len(POS_features[0])):
    n = i
    POS_features_l.append(getsingle(POS_features, n))

In [ ]:
#Get DataFrame
map1 = {'name': names, 'label': labels, 'F_feature': F_features}

for i in range(len(WC_features[0])):
    key = 'WC_' + str(i + 1)
    value = WC_features_l[i]
    map1[key] = value

for i in range(len(GRF_features[0])):
    key = 'GRF_' + str(i + 1)
    value = GRF_features_l[i]
    map1[key] = value

for i in range(len(POS_features[0])):
    key = 'POS_' + str(i + 1)
    value = POS_features_l[i]
    map1[key] = value

allofall = pd.DataFrame(map1)
F_features_u = np.array(F_features)
F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
allofall['F_feature'] = F_features_u

allofall.to_csv('allofall_blogs.csv', index=False)
genderbias = pd.read_csv('blogs_genderbias.csv')
allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
    'word ratio']
allofall.to_csv('blogs_features.csv', index=False)

### 2.2 Novels

In [ ]:
#Word count and feature extract
names = []
F_features = []
GRF_features = []
WC_features = []
labels = []
lengths = []
for gender in [0, 1]:
    if gender == 0:
        txtDir = './5000/female/'
    else:
        txtDir = './5000/male/'

    print("Processing gender: {}".format(txtDir))
    handle_gender = os.listdir(txtDir)
    print("Files:", len(handle_gender))
    for m in handle_gender:
        name = txtDir + m
        text = gettext(name)
        words = nltk.word_tokenize(text)
        tags = nltk.pos_tag(words)
        words_l = wordlemmatize(tags)

        length = len(words)
        F_feature = F_measure(tags)
        GRF_feature = Gender_Preferential_Features(words_l)
        WC_feature = Word_Classes_Feature(words_l)

        names.append(name)
        lengths.append(length)
        F_features.append(F_feature)
        GRF_features.append(GRF_feature)
        WC_features.append(WC_feature)
        labels.append(gender)

WC_features_l = []
GRF_features_l = []

for i in range(len(WC_features[0])):
    n = i
    WC_features_l.append(getsingle(WC_features, n))

for i in range(len(GRF_features[0])):
    n = i
    GRF_features_l.append(getsingle(GRF_features, n))

map1 = {
    'name': names,
    'label': labels,
    'F_feature': F_features,
    'word count': lengths
}

for i in range(len(WC_features[0])):
    key = 'WC_' + str(i + 1)
    value = WC_features_l[i]
    map1[key] = value

for i in range(len(GRF_features[0])):
    key = 'GRF_' + str(i + 1)
    value = GRF_features_l[i]
    map1[key] = value

allofall = pd.DataFrame(map1)

F_features_u = np.array(F_features)
F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
allofall['F_feature'] = F_features_u

allofall.to_csv('allofall_5000.csv', index=False)
genderbias = pd.read_csv('5000_genderbias.csv')
allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
    'word ratio']
allofall.to_csv('5000_features.csv', index=False)

In [ ]:
allofall['word count'].describe()

In [ ]:
#shorten text
length_list = [200, 400, 600, 800, 1200, 1600, 2000, 27850]

for l in length_list:
    for gender in [0, 1]:
        if gender == 0:
            txtDir = './5000/female/'
            saveDir = f'./5000-{l}/female/'
        else:
            txtDir = './5000/male/'
            saveDir = f'./5000-{l}/male/'
        print("Processing gender: {}".format(txtDir))
        handle_gender = os.listdir(txtDir)
        print("Files:", len(handle_gender))
        for idx, m in enumerate(handle_gender):
            name = txtDir + m
            text = gettext(name)
            words = nltk.word_tokenize(text)
            if len(words) < (l + 1):
                continue
            words = words[:l]
            text = ''
            for word in words:
                text = text + word + ' '
            with open(op.join(saveDir, m), 'w') as f:
                f.write(text)
            if (idx + 1) % 500 == 0:
                print('Done:{}'.format(idx + 1))

In [8]:
#feature extract
names = []
F_features = []
GRF_features = []
WC_features = []
labels = []


for l in length_list:
    for gender in [0, 1]:
        if gender == 0:
            txtDir = f'./5000-{l}/female/'
        else:
            txtDir = f'./5000-{l}/male/'

        print("Processing gender: {}".format(txtDir))
        handle_gender = os.listdir(txtDir)
        print("Files:", len(handle_gender))
        for m in handle_gender:
            name = txtDir + m
            text = gettext(name)
            words = nltk.word_tokenize(text)
            tags = nltk.pos_tag(words)
            words_l = wordlemmatize(tags)

            F_feature = F_measure(tags)
            GRF_feature = Gender_Preferential_Features(words_l)
            WC_feature = Word_Classes_Feature(words_l)

            names.append(name)
            F_features.append(F_feature)
            GRF_features.append(GRF_feature)
            WC_features.append(WC_feature)
            labels.append(gender)

    WC_features_l = []
    GRF_features_l = []

    for i in range(len(WC_features[0])):
        n = i
        WC_features_l.append(getsingle(WC_features, n))

    for i in range(len(GRF_features[0])):
        n = i
        GRF_features_l.append(getsingle(GRF_features, n))

    map1 = {'name': names, 'label': labels, 'F_feature': F_features}

    for i in range(len(WC_features[0])):
        key = 'WC_' + str(i + 1)
        value = WC_features_l[i]
        map1[key] = value

    for i in range(len(GRF_features[0])):
        key = 'GRF_' + str(i + 1)
        value = GRF_features_l[i]
        map1[key] = value

    allofall = pd.DataFrame(map1)

    F_features_u = np.array(F_features)
    F_features_u = (F_features_u - np.mean(F_features_u)) / np.std(F_features_u)
    allofall['F_feature'] = F_features_u

    allofall.to_csv(f'allofall_5000-{l}.csv',index = False)
    genderbias = pd.read_csv(f'5000-{l}_genderbias.csv')
    allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias['word ratio']
    allofall.to_csv(f'5000-{l}_features.csv',index = False)

Processing gender: ./5000-1600/female/
Files: 2468
Processing gender: ./5000-1600/male/
Files: 2456


## 3. Train

In [11]:
#train
allf = []
allf_i = []
gb = []
gb_i = []
without = []
without_i = []
featurelist = []

for l in length_list:
    allofall = pd.read_csv(f'5000-{l}_features.csv')
    #genderbias = pd.read_csv(f'5000-{l}_genderbias.csv')
    #allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
    #    'word ratio']
    data, target = allofall.drop(
        columns=['name', 'label']).iloc[:].values, allofall['label'].values
    train_X, test_X, train_y, test_y = train_test_split(data,
                                                        target,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True)
    data, target = pd.concat(
        (pd.DataFrame(train_X), pd.DataFrame(test_X))), pd.concat(
            (pd.DataFrame(train_y), pd.DataFrame(test_y)))
    data1 = data.iloc[:, :-2]
    data2 = data.iloc[:, -2:]
    rnd_clf = RandomForestClassifier(n_estimators=800,
                                     max_leaf_nodes=64,
                                     n_jobs=-1)

    rnd_clf.fit(data.values, target.values.reshape(-1, ))
    scores_rnd_clf_cv = cross_val_score(rnd_clf,
                                        data.values,
                                        target.values.reshape(-1, ),
                                        cv=5)
    print('\n Length:{}'.format(l))
    print('All features:')
    print(scores_rnd_clf_cv)
    print("Accuracy: %0.2f (+/- %0.2f)" %
          (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))
    allf.append(scores_rnd_clf_cv.mean())
    allf_i.append([rnd_clf.feature_importances_])

    rnd_clf.fit(data1.values, target.values.reshape(-1, ))
    scores_rnd_clf_cv = cross_val_score(rnd_clf,
                                        data1.values,
                                        target.values.reshape(-1, ),
                                        cv=5)
    print('Without Gender bias:')
    print(scores_rnd_clf_cv)
    print("Accuracy: %0.2f (+/- %0.2f)" %
          (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))
    without.append(scores_rnd_clf_cv.mean())
    without_i.append([rnd_clf.feature_importances_])

    rnd_clf.fit(data2.values, target.values.reshape(-1, ))
    scores_rnd_clf_cv = cross_val_score(rnd_clf,
                                        data2.values,
                                        target.values.reshape(-1, ),
                                        cv=5)
    print('Gender bias:')
    print(scores_rnd_clf_cv)
    print("Accuracy: %0.2f (+/- %0.2f)" %
          (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))
    gb.append(scores_rnd_clf_cv.mean())
    gb_i.append([rnd_clf.feature_importances_])


 Length:2000
All features:
[0.75914634 0.75788403 0.74439919 0.74541752 0.76374745]
Accuracy: 0.75 (+/- 0.02)
Without Gender bias:
[0.69613821 0.70803662 0.68940937 0.71181263 0.71384929]
Accuracy: 0.70 (+/- 0.02)
Gender bias:
[0.70833333 0.7456765  0.71792261 0.70875764 0.71995927]
Accuracy: 0.72 (+/- 0.03)


In [30]:
#length vs accuracy
df = pd.DataFrame({'length':length_list,
                  'All features':allf,
                   'All features_importances':allf_i_new,
                  'Without gb':without,
                   'Without gb_importances':without_i_new,
                  'Gender bias':gb,
                  'Gender bias_importances':gb_i_new})
df = df.set_index('length')
#df.to_csv('top5_sum.csv')

## 4. Others

In [ ]:
#10wincv
from sklearn.model_selection import cross_val_score

for i in range(1, 11):
    fname = str(10 + i) + '_win_5000_genderbias.csv'
    genderbias = pd.read_csv(fname)
    allofall['bias'], allofall['word ratio'] = genderbias['bias'], genderbias[
        'word ratio']
    data, target = allofall.drop(columns=['name', 'label', 'word count']
                                 ).iloc[:].values, allofall['label'].values
    train_X, test_X, train_y, test_y = train_test_split(data,
                                                        target,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True)
    data, target = pd.concat(
        (pd.DataFrame(train_X), pd.DataFrame(test_X))), pd.concat(
            (pd.DataFrame(train_y), pd.DataFrame(test_y)))

    rnd_clf = RandomForestClassifier(n_estimators=1000,
                                     max_leaf_nodes=50,
                                     n_jobs=-1)
    scores_rnd_clf_cv = cross_val_score(rnd_clf,
                                        data.values,
                                        target.values.reshape(-1, ),
                                        cv=5)
    print('Windows:%.2f'%(i))
    print(scores_rnd_clf_cv)
    print("Accuracy: %0.2f (+/- %0.2f)" %
          (scores_rnd_clf_cv.mean(), scores_rnd_clf_cv.std() * 2))

In [68]:
m = {'F_feature': 0.06774622830146265, 'WC_1': 0.030750302311476855, 'WC_2': 0.05194608467327763, 'WC_3': 0.04181383957225258, 'WC_4': 0.042101931752933544, 'WC_5': 0.01859543308878391, 'WC_6': 0.006798851770971471, 'WC_7': 0.029499780289836085, 'WC_8': 0.016370913051504194, 'WC_9': 0.017484348718995835, 'WC_10': 0.027083945643367267, 'WC_11': 0.023206040153359947, 'WC_12': 0.01985475337506297, 'WC_13': 0.027982606161854492, 'WC_14': 0.0198331507264329, 'WC_15': 0.07211366178386407, 'WC_16': 0.004959599670063322, 'WC_17': 0.04699624733726917, 'WC_18': 0.0103817547431019, 'WC_19': 0.008637290754813388, 'WC_20': 0.02385450240258374, 'WC_21': 0.017809444504190624, 'WC_22': 0.014857432338504355, 'WC_23': 0.01861932381502889, 'GRF_1': 0.00984362622705038, 'GRF_2': 0.1188716568744626, 'GRF_3': 0.07429039446230534, 'GRF_4': 0.006839419658838325, 'GRF_5': 0.01373370302145265, 'GRF_6': 0.014094758264402284, 'GRF_7': 0.007359464148519862, 'GRF_8': 0.07731371783211263, 'GRF_9': 0.0165323457001773, 'GRF_10': 0.0018234468696868434}
m = sorted(m.items(),key=lambda x:x[1],reverse=True)
m = m[:5]
ms.append(m)

In [79]:
name = allofall.columns[2:]

In [84]:
i = without_i[1]
handle = [0]*5
handle[0]= i[0][0]
handle[1]=sum(i[0][1:24])
handle[2]=sum(i[0][24:34])
handle[3:]=i[0][-2:]

In [83]:
name = ['F','WC','GRF','bias','word ratio']

In [100]:
sum33 = []
for i in sum2:
    sum33.append(list(zip(flist,i)))

In [49]:
df.insert(2,'All features sum_importances',sum11)
df.insert(5,'Without gb sum_importances',sum22)

In [102]:
df.iloc[:,2] = sum33

In [71]:
sum22 = []
for i in sum33:
    sum22.append(list(sorted(i,key=lambda x:x[1],reverse=True))[:5])